# Assignment 4: Wheres Waldo?
### Name: Eileanor LaRocco
In this assignment, you will develop an object detection algorithm to locate Waldo in a set of images. You will develop a model to detect the bounding box around Waldo. Your final task is to submit your predictions on Kaggle for evaluation.

### Imports

In [1]:
import os
import opendatasets as od
import pandas as pd
import numpy as np
import random
import csv
import matplotlib.pyplot as plt
#!pip install opencv-python
import cv2

import shutil
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import DataLoader
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image

In [2]:
SEED = 1

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = device = torch.device("cuda") #mps
print(device)

cuda


### Download Data

In [3]:
od.download('https://www.kaggle.com/competitions/2024-fall-ml-3-hw-4-wheres-waldo/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  eileanorplarocco


Your Kaggle Key:

  ········


100%|██████████| 38.2M/38.2M [00:00<00:00, 57.8MB/s]


Extracting archive ./2024-fall-ml-3-hw-4-wheres-waldo/2024-fall-ml-3-hw-4-wheres-waldo.zip to ./2024-fall-ml-3-hw-4-wheres-waldo


### Paths

In [4]:
train_folder = "2024-fall-ml-3-hw-4-wheres-waldo/train/train" # Original Train Images
test_folder = "2024-fall-ml-3-hw-4-wheres-waldo/test/test" # Original Test Images
annotations_file = "2024-fall-ml-3-hw-4-wheres-waldo/annotations.csv" # Original Annotations File

# Preprocessing

### Draw bounding boxes on each training image to check accuracy

In [5]:
# Paths
output_folder = "2024-fall-ml-3-hw-4-wheres-waldo/checks"  # Folder to save images with drawn boxes

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Read the CSV file
# Assumes the CSV columns are: filename, xmin, ymin, xmax, ymax
annotations = pd.read_csv(annotations_file)

# Iterate through each image in the annotations
for _, row in annotations.iterrows():
    image_name = row["filename"]
    x_min, y_min, x_max, y_max = row["xmin"], row["ymin"], row["xmax"], row["ymax"]
    
    # Load the image
    image_path = os.path.join(train_folder, image_name)
    if not os.path.exists(image_path):
        print(f"Image {image_path} not found. Skipping...")
        continue
    image = cv2.imread(image_path)
    
    # Draw the bounding box
    # cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (B, G, R), thickness)
    cv2.rectangle(image, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 4)
    
    # Optionally, add a label or text
    label = "Waldo"
    cv2.putText(image, label, (int(x_min), int(y_min) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Save the image
    output_path = os.path.join(output_folder, image_name)
    cv2.imwrite(output_path, image)

    print(f"Annotated image saved to {output_path}")

print("All bounding boxes have been drawn and saved.")

Annotated image saved to 2024-fall-ml-3-hw-4-wheres-waldo/checks/1.jpg
Annotated image saved to 2024-fall-ml-3-hw-4-wheres-waldo/checks/10.jpg
Annotated image saved to 2024-fall-ml-3-hw-4-wheres-waldo/checks/11.jpg
Annotated image saved to 2024-fall-ml-3-hw-4-wheres-waldo/checks/12.jpg
Annotated image saved to 2024-fall-ml-3-hw-4-wheres-waldo/checks/13.jpg
Annotated image saved to 2024-fall-ml-3-hw-4-wheres-waldo/checks/14.jpg
Annotated image saved to 2024-fall-ml-3-hw-4-wheres-waldo/checks/15.jpg
Annotated image saved to 2024-fall-ml-3-hw-4-wheres-waldo/checks/17.jpg
Annotated image saved to 2024-fall-ml-3-hw-4-wheres-waldo/checks/18.jpg
Annotated image saved to 2024-fall-ml-3-hw-4-wheres-waldo/checks/19.jpg
Annotated image saved to 2024-fall-ml-3-hw-4-wheres-waldo/checks/2.jpg
Annotated image saved to 2024-fall-ml-3-hw-4-wheres-waldo/checks/3.jpg
Annotated image saved to 2024-fall-ml-3-hw-4-wheres-waldo/checks/4.jpg
Annotated image saved to 2024-fall-ml-3-hw-4-wheres-waldo/checks/5.j

In [6]:
### Remove training images that don't include waldo as he appears in test images
#16,22-27 do not include waldo in a background, 8 and 10 include waldo in the postcard

def delete_files_from_list(folder_path, file_names_to_delete):
    """Deletes files in the given folder if their name is in the provided list."""

    for filename in os.listdir(folder_path):
        if filename in file_names_to_delete:
            file_path = os.path.join(folder_path, filename)
            os.remove(file_path)
            print(f"Deleted: {file_path}")

if __name__ == "__main__":
    folder_path =  "2024-fall-ml-3-hw-4-wheres-waldo/train/train"
    file_names_to_delete = ["8.jpg", "10.jpg", "16.jpg", "22.jpg", "23.jpg", "24.jpg", "25.jpg", "26.jpg", "27.jpg"]
    delete_files_from_list(folder_path, file_names_to_delete)

Deleted: 2024-fall-ml-3-hw-4-wheres-waldo/train/train/24.jpg
Deleted: 2024-fall-ml-3-hw-4-wheres-waldo/train/train/22.jpg
Deleted: 2024-fall-ml-3-hw-4-wheres-waldo/train/train/10.jpg
Deleted: 2024-fall-ml-3-hw-4-wheres-waldo/train/train/27.jpg
Deleted: 2024-fall-ml-3-hw-4-wheres-waldo/train/train/25.jpg
Deleted: 2024-fall-ml-3-hw-4-wheres-waldo/train/train/8.jpg
Deleted: 2024-fall-ml-3-hw-4-wheres-waldo/train/train/23.jpg
Deleted: 2024-fall-ml-3-hw-4-wheres-waldo/train/train/16.jpg
Deleted: 2024-fall-ml-3-hw-4-wheres-waldo/train/train/26.jpg


In [10]:
# Remove associated annotations
import pandas as pd

df = pd.read_csv(annotations_file)

# Keep rows where the value in column 'filename' is not in the list
df = df[~df['filename'].isin(file_names_to_delete)]
df.to_csv(annotations_file, index=False)


In [17]:
# Create more training data - randomly crop 512x512 sections where full waldo bounding box exists

import os
import pandas as pd
import csv
import random
from PIL import Image

def generate_crops(train_folder, annotations_file, output_folder, output_csv, crop_size=512, num_crops=5):
    # Ensure the output folder exists - if not, make it
    os.makedirs(output_folder, exist_ok=True)
    
    # Load annotations
    annotations = pd.read_csv(annotations_file)
    
    new_annotations = []
    
    for _, row in annotations.iterrows():
        img_path = os.path.join(train_folder, row['filename'])
        image = Image.open(img_path)
        width, height = image.size
        
        x_min, y_min, x_max, y_max = row['xmin'], row['ymin'], row['xmax'], row['ymax']
        
        for i in range(num_crops):
            # Generate random crop coordinates
            crop_x_min = max(0, random.randint(x_min - crop_size, x_min))
            crop_y_min = max(0, random.randint(y_min - crop_size, y_min))
            crop_x_max = min(crop_x_min + crop_size, width)
            crop_y_max = min(crop_y_min + crop_size, height)
            
            # Adjust crop to ensure it's exactly crop_size x crop_size
            if crop_x_max - crop_x_min < crop_size:
                crop_x_min = max(0, crop_x_max - crop_size)
            if crop_y_max - crop_y_min < crop_size:
                crop_y_min = max(0, crop_y_max - crop_size)
            
            # Ensure Waldo is fully included in the crop
            if not (crop_x_min <= x_min and crop_x_max >= x_max and crop_y_min <= y_min and crop_y_max >= y_max):
                continue
            
            # Crop the image
            cropped_image = image.crop((crop_x_min, crop_y_min, crop_x_max, crop_y_max))
            
            # Save the cropped image
            new_filename = f"{os.path.splitext(row['filename'])[0]}_crop_{i}.jpg"
            cropped_image.save(os.path.join(output_folder, new_filename))
            
            # Calculate the new bounding box coordinates
            new_x_min = x_min - crop_x_min
            new_y_min = y_min - crop_y_min
            new_x_max = x_max - crop_x_min
            new_y_max = y_max - crop_y_min
            
            # Add to new annotations
            new_annotations.append({
                "filename": new_filename,
                "xmin": new_x_min,
                "ymin": new_y_min,
                "xmax": new_x_max,
                "ymax": new_y_max
            })
    
    # Save new annotations to CSV
    new_annotations_df = pd.DataFrame(new_annotations)
    new_annotations_df.to_csv(output_csv, index=False)
    print(f"Generated crops saved to {output_folder}. New annotations saved to {output_csv}.")

# Define the paths
output_folder = "2024-fall-ml-3-hw-4-wheres-waldo/train/chunks"
output_csv = "2024-fall-ml-3-hw-4-wheres-waldo/chunks_annotations.csv"
crop_size = 512
num_crops = 40

#Use function
generate_crops(train_folder, annotations_file, output_folder, output_csv, crop_size, num_crops)


Generated crops saved to 2024-fall-ml-3-hw-4-wheres-waldo/train/chunks. New annotations saved to 2024-fall-ml-3-hw-4-wheres-waldo/chunks_annotations.csv.


# Train/Test Split

In [18]:
# Split training data into train and validation sets
annotations = pd.read_csv("2024-fall-ml-3-hw-4-wheres-waldo/chunks_annotations.csv")
image_files = annotations["filename"].unique()
train_images, val_images = train_test_split(image_files, test_size=0.2, random_state=42)

def filter_csv_by_column(input_csv, output_csv, column_name, values_list):

    # Load the CSV into a DataFrame
    df = pd.read_csv(input_csv)

    # Filter the DataFrame
    filtered_df = df[df[column_name].isin(values_list)]

    # Save the filtered DataFrame to a new CSV file
    filtered_df.to_csv(output_csv, index=False)

#Train Annotations
values_list = list(train_images)
output_csv = "2024-fall-ml-3-hw-4-wheres-waldo/train_annotations.csv"
column_name = "filename"
filter_csv_by_column("2024-fall-ml-3-hw-4-wheres-waldo/chunks_annotations.csv", output_csv, column_name, values_list)

#Test Annotations
values_list = list(val_images)
output_csv = "2024-fall-ml-3-hw-4-wheres-waldo/test_annotations.csv"
column_name = "filename"
filter_csv_by_column("2024-fall-ml-3-hw-4-wheres-waldo/chunks_annotations.csv", output_csv, column_name, values_list)

#Train/Test Split (80/20)
def split_directory(source_dir, target_dir, file_list):
    """Splits files from source_dir to target_dir based on file_list."""

    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    for file_name in file_list:
        source_path = os.path.join(source_dir, file_name)
        target_path = os.path.join(target_dir, file_name)

        if os.path.exists(source_path):
            shutil.move(source_path, target_path)
            #print(f"Moved: {file_name}")
        else:
            print(f"File not found: {file_name}")

if __name__ == "__main__":
    source_dir = "2024-fall-ml-3-hw-4-wheres-waldo/train/chunks"
    target_dir = "2024-fall-ml-3-hw-4-wheres-waldo/train/val"
    file_list = list(val_images)

    split_directory(source_dir, target_dir, file_list)